# 1 - get node dicts

In [1]:
import sys  
from os import path
import numpy as np
import pandas as pd

sys.path.append('../src/')


In [2]:
from modules.classes import FEBio_xml_parser as FEBIO_PARSER

#### Using functions from "get_node_dicts"

In [3]:
def get_nodes_from_elems(elems):
    """Creates a dictionary with nodes as keys and a list of element numbers for each node."""
    nodeDict = dict()
    for el in elems:
        el_num = int(el[0])
        el_nods = [int(v) for v in el[1:]]
        for node_num in el_nods:
            if node_num in nodeDict:
                nodeDict[node_num].append(el_num)
            else:
                nodeDict[node_num] = [el_num]
    return nodeDict

In [4]:
def get_faceSets(nodeLists):
    """Returns a list of sets containing node numbers of a given face"""
    to_return = list()
    for _list in nodeLists:
        to_return.append(set([int(v[0]) for v in _list]))
    return to_return

In [5]:
def get_face_dict(nodeDict, faceSets):
    """Returns list with dicts containing the intersections between a nodeDict and a faceSet"""
    to_return = list()
    for _faceSet in faceSets:
        faceDict = dict()
        for node_num in _faceSet:
            faceDict[node_num] = nodeDict[node_num]
        to_return.append(faceDict)
    return to_return

In [6]:
file_dir = "D:/Igor/Research_USF/University of South Florida/Mao, Wenbin - Igor/Febio-Models/Active-Models/PAQ/Myo_test/myo_hex_coarse_8_PAQ"
file_name = "myo_hex_coarse_8_PAQ.feb"


file_path = path.join(file_dir, file_name)

In [7]:
FBEIO_obj = FEBIO_PARSER.FEBio_xml_parser(file_path)
FEBio_nodests = FBEIO_obj.get_geometry_data(what=[("NodeSet", "Endocardio", "node"), ("NodeSet", "Epicardio", "node"), ("NodeSet", "Top_surface", "node")])

nodes = FEBio_nodests[0]
elems = FEBio_nodests[1]
endoNodeList = FEBio_nodests[2]["Endocardio"]
epiNodeList = FEBio_nodests[3]["Epicardio"]
topNodeList = FEBio_nodests[4]["Top_surface"]

Initializing...
-Parsing.
-Rooting.
-Finding tags.
--Found: Module.
--Found: Control.
--Found: Material.
--Found: Globals.
--Found: Geometry.
--Found: Boundary.
--Found: Loads.
--Found: LoadData.
--Found: Output.
--Found: MeshData.


In [8]:
nodeDict = get_nodes_from_elems(elems)
faceSets = get_faceSets([endoNodeList, epiNodeList, topNodeList])
faceDicts = get_face_dict(nodeDict, faceSets)
(endoFace, epiFace, topFace) = faceDicts

# 2 - Compute nodal values

In [9]:
from os import listdir, makedirs
from os.path import isfile, join, isdir

def find_files(path_to_folder, condition=None):
	if (condition):
		if (condition[0] == "fileFormat"):
			_l = len(condition[1])
			return [(join(path_to_folder,f), f,f[:-_l - 1]) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f)) and f[-_l:] == condition[1]]
		else:
			raise(ValueError("Condition should be a tuple where the first argument is the condition name and the second is its value"))
	else:
		return [(join(path_to_folder,f), f, f.rsplit(".")[0]) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f))]

### Load dataframes

In [10]:
# import compiled data

compiled_dir = "./compiled_data"
elems_file = "elems_compiled.pickle"
nodes_file = "nodes_compiled.pickle"

e_p = join(compiled_dir, elems_file)
n_p = join(compiled_dir, nodes_file)

elems_df = pd.read_pickle(e_p)
nodes_df = pd.read_pickle(n_p)

### Create a dictionary with the elem number and its data

In [11]:
elem_vec = elems_df.to_dict('records')
elem_data_dict = dict()
for elem in elem_vec:
    elem_data_dict[(elem["elem"],elem["time"])] = elem

### Calculate nodal data based on average values of neighboring nodes

In [12]:
elem_data_labels = ["sx", "sy", "sz", "sxy", "sxz", "syz"]

# vectorize nodes_df
nodes_vec = nodes_df.to_dict('records')

# loop through nodes and add nodal data based on elemen value
for node in nodes_vec:
    # get node number and timestep
    node_num = node["node"]
    time_step = node["time"]

    # get elems that are connected to given node
    elems_c_node = nodeDict[int(node_num)]

    # get elem_data
    elem_data = np.zeros((1,len(elem_data_labels)))
    for elem_num in elems_c_node:
        elem_vals = elem_data_dict[(elem_num, time_step)]
        elem_data += np.array([elem_vals[v] for v in elem_data_labels])
    elem_data = elem_data / len(elems_c_node)

    # add nodal data
    for i, el_label in enumerate(elem_data_labels):
        node[el_label] = elem_data[0][i]

### Recreate nodal dataframe with additional data

In [13]:
new_nodes_df = pd.DataFrame.from_records(nodes_vec)


In [14]:
new_nodes_df

,time,node,x,y,z,ux,uy,uz,fileName,sx,sy,sz,sxy,sxz,syz
0,0.004,1.0,7.532080e-04,3.288430e-03,-63.3090,0.000753,0.003288,1.691010,myo_hex_coarse_8_PAQ,-4.479295,-4.487203,-2.370364,-0.006004,-0.003095,0.000133
1,0.004,2.0,2.143130e-15,-5.249160e-31,-74.7245,0.000000,0.000000,0.275538,myo_hex_coarse_8_PAQ,-2.541123,-2.536295,-4.239592,0.103267,-0.007552,0.007496
2,0.004,3.0,3.170800e+01,-3.688090e+00,20.0000,-2.024590,-3.688090,0.000000,myo_hex_coarse_8_PAQ,0.153580,0.851263,2.911360,-0.343081,-0.064210,0.710516
3,0.004,4.0,2.160600e+01,1.375530e+00,20.0000,-2.181190,1.375530,0.000000,myo_hex_coarse_8_PAQ,0.040380,-0.045753,0.939052,-0.198804,-0.016145,-0.297368
4,0.004,5.0,3.856660e-03,1.302220e-03,-65.0116,0.003857,0.001302,1.238390,myo_hex_coarse_8_PAQ,-3.805604,-3.815178,-2.588628,-0.004874,-0.000586,0.000595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45013,0.200,22505.0,2.452680e+01,-2.234780e+01,17.6554,-1.966900,-3.099000,-0.211331,myo_hex_coarse_8_PAQ,2.113051,12.271661,27.291750,6.378113,6.002079,8.097112
45014,0.200,22506.0,2.713600e+01,-1.909560e+01,17.6514,-1.561350,-3.319080,-0.215368,myo_hex_coarse_8_PAQ,0.670111,14.139651,28.194638,5.083924,5.206872,9.393216
45015,0.200,22507.0,2.931470e+01,-1.554540e+01,17.6564,-1.133710,-3.490040,-0.210332,myo_hex_coarse_8_PAQ,-0.525584,15.509006,28.673550,3.308281,4.059597,10.181537
45016,0.200,22508.0,3.102770e+01,-1.174320e+01,17.6467,-0.691526,-3.599090,-0.220055,myo_hex_coarse_8_PAQ,-1.041410,15.984050,28.272125,1.234811,2.649293,10.408104


In [15]:
compiled_dir = "./compiled_data"
new_nodes_file = "nodes_with_data.pickle"

new_nodes_df.to_pickle(join(compiled_dir, new_nodes_file))